In [15]:

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import os
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [2]:

# extracting from pdf and splitting it 
loader = PyPDFLoader("lab_values.pdf")
doc = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
document = splitter.split_documents(documents=doc)

In [9]:
#storing in vector db
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("hugging-face-api")
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",)

db = Chroma(
    persist_directory="mydb",
    collection_name="my_collection",
    embedding_function=embedding
)
retriever = db.as_retriever(search_kwargs={'k':3})

In [ ]:
# Making pipline
# llm = HuggingFaceEndpoint(
#     repo_id="google/flan-t5-large",
#     max_new_tokens=512,
#     temperature=0.3,
#     task="text2text-generation"
# )
llm = ChatGoogleGenerativeAI(api_key=os.getenv('gemini_key'),model="gemini-2.0-flash")
# qa = RetrievalQA.from_chain_type(
#     retriever = retriever,
#     llm = llm,
#     chain_type="map_reduce",
#     return_source_documents=True
# )

In [16]:
# Making custom pipline
prompt = PromptTemplate(
    input_variables=["patient_report","context"],
    template="""
You are a medical assistant.

Here is a patient's lab report:

{patient_report}

Here is reference context from clinical guidelines:

{context}

Analyze the lab report. Explain the abnormal results, and suggest next steps according to standard medical practice.

"""
)

qa_chain = LLMChain(llm=llm,prompt=prompt)

C:\Users\acer\AppData\Local\Temp\ipykernel_16256\400608049.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm,prompt=prompt)


In [11]:
report_loader = PyPDFLoader("sample_lab_report.pdf")
report = report_loader.load()
report_text ="".join([docs.page_content for docs in report])

In [17]:
lab_docs = retriever.get_relevant_documents(report_text)
context = "\n\n".join([lab_doc.page_content for lab_doc in lab_docs])

C:\Users\acer\AppData\Local\Temp\ipykernel_16256\399706216.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  lab_docs = retriever.get_relevant_documents(report_text)


In [18]:
result = qa_chain.run({"patient_report":report_text,"context":context})

C:\Users\acer\AppData\Local\Temp\ipykernel_16256\3666711908.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run({"patient_report":report_text,"context":context})


In [20]:
print(result)

Okay, here's an analysis of John Doe's lab report, highlighting the abnormal results and suggesting potential next steps based on standard medical practice:

**Analysis of Abnormal Lab Results:**

*   **Hemoglobin (Hb): 10.2 g/dL (Low)** - This indicates anemia, meaning John Doe has a lower than normal level of hemoglobin in his blood. Hemoglobin is crucial for carrying oxygen throughout the body.

*   **WBC Count: 12.5 x10^9/L (High)** - This indicates leukocytosis, meaning John Doe has a higher than normal white blood cell count. This often suggests an infection, inflammation, or other immune response.

*   **Glucose (Fasting): 145 mg/dL (High)** - This indicates hyperglycemia, meaning John Doe has a higher than normal fasting blood glucose level. According to the provided reference, a fasting plasma glucose (FPG) of ≥ 126 mg/dL is diagnostic of diabetes. John Doe's result of 145 mg/dL meets this criterion.

*   **ALT (SGPT): 65 U/L (High)** - This indicates elevated liver enzymes. A